# 01_etl_stage_to_raw
---
Este notebook executa a preparação do projeto para o processo `ETL`, transferindo os dados da pasta **Stage** para a **Raw**. Englobando o download (opcional), carga de dados, conversão e movimentação dos arquivos necessários para a continuidade do pipeline.


In [1]:
# Parameters

stage_path = "/opt/airflow/data-layer/raw/stage"
raw_path = "/opt/airflow/data-layer/raw"

postgres_conn_id = "AIRFLOW_VAR_POSTGRES_CONN_ID"


In [ ]:
import os
from datetime import datetime
from pathlib import Path

from pyspark.sql import DataFrame, SparkSession

from transformer.utils.file_io import check_files_in_folder, delete_files, move_files
from transformer.utils.logger import get_logger
from transformer.utils.quality_gates_raw import run_quality_gates_raw
from transformer.utils.spark_helpers import get_spark_session, load_to_postgres


## Job 0: kaggle_download_and_prepare

Este job executa o download, a descompactação e a preparação dos arquivos para a camada Stage. Sua execução permanece desativada porque depende de configurações manuais de acesso à API do Kaggle. Reative as células e informe suas credenciais caso deseje executá-lo.


In [7]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
import numpy as np
import pandas as pd

log = get_logger("kaggle_and_prepare")

NUM_CHUNKS=10


### Definindo funções de download e de divisão

In [8]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
def download_kaggle_dataset(dataset_name: str, dest_path: Path) -> None:
    """
    Faz o download e a extração de um dataset do Kaggle, caso ainda não exista localmente.

    Args:
        dataset_name (str): Nome do dataset no Kaggle (ex: 'usdot/flight-delays').
        dest_path (Path): Caminho destino para os arquivos extraídos.

    Raises:
        Exception: Para erros de autenticação ou rede.
    """
    try:
        existing_files = os.listdir(dest_path)
        has_csv_files = any(file.endswith(".csv") for file in existing_files)

        if has_csv_files:
            log.info(f"[Kaggle] Arquivos já existem em '{dest_path}'. Pulando download.")
            return

        log.info(f"[Kaggle] Nenhum arquivo CSV encontrado. Baixando o dataset '{dataset_name}'.")
        kaggle.api.dataset_download_files(dataset_name, path=dest_path, unzip=True)
        log.info("[Kaggle] Download e extração concluídos com sucesso.")
    except Exception as e:
        log.exception(f"[Kaggle] Ocorreu um erro! Verifique se a API do Kaggle está configurada: {e}.")
        log.exception(f"[Kaggle] Erro: {e}.")
        raise

def split_main_file(source_file: Path, num_chunks: int = NUM_CHUNKS) -> None:
    """
    Divide o arquivo flights.csv em múltiplos chunks de tamanho aproximadamente igual.

    Args:
        source_file (Path): Caminho do arquivo csv principal.
        num_chunks (int): Número de partes a gerar.

    Raises:
        FileNotFoundError: Se o arquivo não existir.
        ValueError: Se o arquivo estiver vazio.
        Exception: Para erros inesperados durante a divisão.
    """
    if not source_file.exists():
        raise FileNotFoundError(f"[Landing] Arquivo não encontrado: '{source_file}'.")

    try:
        df = pd.read_csv(source_file)
        if df.empty:
            raise ValueError(f"O arquivo '{source_file}' está vazio.")

        log.info(f"[Split] Dividindo '{source_file.name}' em {num_chunks} partes aproximadamente iguais.")

        chunks = np.array_split(df, num_chunks)

        for i, chunk_df in enumerate(chunks, start=1):
            chunk_name = f"{source_file.stem}_part_{i:02d}{source_file.suffix}"
            chunk_path = source_file.parent / chunk_name
            chunk_df.to_csv(chunk_path, index=False)
            log.info(f"[Split] Chunk salvo: '{chunk_name}', {len(chunk_df)} linhas.")

        log.info("[Split] Divisão do arquivo concluída com sucesso.")

    except Exception as e:
        log.error(f"[Split] Erro ao dividir o arquivo: {e}.")
        raise


### Runner para o job `kaggle_download_and_prepare`

In [9]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
try:
    # Download
    download_kaggle_dataset(DATASET_NAME, stage_path)

    # Divisão
    flights_file = Path(stage_path) / "flights.csv"
    if flights_file.exists():
        split_main_file(flights_file, num_chunks=NUM_CHUNKS)
    else:
        log.warning(f"[Kaggle] Arquivo 'flights.csv' não encontrado. Verifique a pasta.")

except Exception as e:
    log.error(f"[Kaggle] Falha no processo de preparação: {e}")


## Job 1: load_raw_dbt_data

Este job carrega os **dados brutos** dos arquivos `airlines.csv`, `airports.csv` e `flights_part_1.csv` da pasta **Stage** para o schema `dbt_raw`, que servirá de base para as transformações realizadas pelo dbt. A opção por carregar apenas o arquivo `flights_part_1.csv` na tabela de voos foi tomada por motivos de performance.

In [ ]:
log = get_logger("load_dbt_data")
spark = get_spark_session("load_dbt_data")

log.info("[dbt][Raw] Sessão Spark iniciada.")


### Runner para o job `load_dbt_data`

In [ ]:
try:
    log.info("[dbt][Raw] Iniciando job de carga dos dados para o dbt.")

    read_options = {
        "header": "true",
        "inferSchema": "false",
        "delimiter": ",",
    }

    # Carregar airlines.csv (RAW via Spark)
    try:
        airlines_path = str(Path(stage_path) / "airlines.csv")

        log.info(f"[dbt][Raw] Lendo airlines: {airlines_path}.")

        airlines_df = spark.read.options(**read_options).csv(airlines_path)

        log.info("[dbt][Raw] Inserindo dados em 'dbt_raw.airlines'.")
        load_to_postgres(
            df=airlines_df,
            db_conn_id=postgres_conn_id,
            table_name="dbt_raw.airlines",
            mode="overwrite",
        )

    except Exception as e:
        log.exception(f"[dbt][Raw][Error] Falha ao carregar airlines: {e}.")
        raise

    # Carregar airports.csv
    try:
        airports_path = str(Path(stage_path) / "airports.csv")

        log.info(f"[dbt][Raw] Lendo airports: {airports_path}.")

        airports_df = spark.read.options(**read_options).csv(airports_path)

        log.info("[dbt][Raw] Inserindo dados em 'dbt_raw.airports'.")
        load_to_postgres(
            df=airports_df,
            db_conn_id=postgres_conn_id,
            table_name="dbt_raw.airports",
            mode="overwrite",
        )

    except Exception as e:
        log.exception(f"[dbt][Raw][Error] Falha ao carregar airports: {e}.")
        raise

    # Carregar flights_part_03.csv
    try:
        flights_chunk_path = str(Path(stage_path) / "flights_part_01.csv")

        log.info(f"[dbt][Raw] Lendo chunk de flights: {flights_chunk_path}.")

        flights_df = spark.read.options(**read_options).csv(flights_chunk_path)

        log.info("[dbt][Raw] Inserindo dados em 'dbt_raw.flights'.")
        load_to_postgres(
            df=flights_df,
            db_conn_id=postgres_conn_id,
            table_name="dbt_raw.flights",
            mode="overwrite",
        )

    except Exception as e:
        log.exception(f"[dbt][Raw][Error] Falha ao carregar flights: {e}.")
        raise

    log.info("[dbt][Raw] Carga concluída com sucesso.")

except Exception as e:
    log.exception(f"[dbt][Raw][Error] Falha na execução do job: {e}.")
    raise

finally:
    log.info("[dbt][Raw] Job de carga dos dados para o dbt encerrado.")


## Job 2: unify_flight_chunks

Este job realiza a unificação dos arquivos `flights_part_*.csv` presentes na pasta **Stage**, consolidando-os em um único arquivo Parquet `flights.parquet`.

In [ ]:
log = get_logger("unify_chunks")

spark = get_spark_session("UnifyFlightChunks")
log.info("[UnifyChunks] SparkSession iniciada.")

# Ajustes de performance para o Spark
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "32")


### Definindo função de unificação

In [11]:
def reassemble_chunks(spark: SparkSession, chunk_files: list[str], header: bool = True) -> DataFrame:
    """
    Lê múltiplos arquivos de chunk (flights_part_*.csv) e os unifica em um único DataFrame Spark.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        chunk_files (list[str]): Lista de caminhos completos dos arquivos csv a serem unificados.
        header (bool, opcional): Define se os arquivos csv possuem cabeçalho. Padrão: True.

    Returns:
        DataFrame: DataFrame Spark consolidado com todos os chunks.
    """
    if not chunk_files:
        raise ValueError("[UnifyChunks][Error] Nenhum arquivo de chunk fornecido para unificação.")

    log.info(f"[UnifyChunks] Lendo e concatenando {len(chunk_files)} arquivo(s) de chunk.")

    try:
        df = (
            spark.read
            .option("header", header)
            .option("inferSchema", False)
            .csv(chunk_files)
        )
        
        log.info("[UnifyChunks] Unificação dos chunks concluída com sucesso.")
        
        return df

    except Exception as e:
        log.error(f"[UnifyChunks][Error] Falha ao ler os arquivos csv: {e}.")
        
        raise


### Runner para o job `unify_flight_chunks`

In [ ]:
try:
    log.info("[UnifyChunks] Iniciando job de unificação de chunks.")

    # Localiza arquivos csv na stage
    csv_files = check_files_in_folder(stage_path, "*.csv")
    chunk_files = [f for f in csv_files if "flights_part" in f]

    if not chunk_files:
        raise FileNotFoundError(f"[UnifyChunks][Error] Nenhum arquivo de chunk encontrado em {stage_path}.")

    # Unifica os chunks
    df_unified = reassemble_chunks(spark, chunk_files)

    # Executa quality gates
    required_columns = [
        'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 
        'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 
        'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 
        'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 
        'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 
        'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
    ]
    run_quality_gates_raw(df_unified, "raw_flights", required_columns)

    # Salva o arquivo unificado
    stage_output = f"{stage_path}/flights.parquet"
    df_unified.write.mode("overwrite").option("compression", "snappy").parquet(stage_output)
    
    log.info(f"[UnifyChunks] Arquivo unificado salvo em: {stage_output}.")

except Exception as e:
    log.exception(f"[UnifyChunks][Error] Falha durante execução: {e}")
    raise

finally:
    log.info("[UnifyChunks] Job de unificação de chunks encerrado.")


In [ ]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[UnifyChunks] Sessão Spark finalizada.")


## Job 3: convert_csv_to_parquet

Este job realiza a conversão dos arquivos csv presentes na pasta **Stage** (exceto os chunks `flights_part_*`) para o formato **parquet**, mantendo o schema original dos dados.


In [ ]:
log = get_logger("convert_csv")

spark = get_spark_session("ConvertCsvToParquet")
log.info("[ConvertCSV] SparkSession iniciada.")

# Ajustes de performance para o Spark
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "32")


### Definindo função de conversão

In [15]:
def convert_csv_to_parquet(spark: SparkSession, csv_files: list[str], stage_path: str) -> None:
    """
    Converte arquivos csv em formato parquet.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        csv_files (list[str]): Lista de caminhos dos arquivos csv a converter.
        stage_path (str): Caminho base da camada Stage.
    """
    if not csv_files:
        raise ValueError("[ConvertCSV][Error] Nenhum arquivo csv fornecido para conversão.")

    for csv in csv_files:
        try:
            # Resolve caminhho do arquivo
            base_name = os.path.basename(csv).replace(".csv", ".parquet")

            log.info(f"[ConvertCSV] Lendo arquivo csv: {csv}.")

            df = (
                spark.read
                .option("header", True)
                .option("inferSchema", False)
                .csv(csv)
            )

            # Executa quality gates
            if df.columns[1] == 'AIRLINE':
                required_columns = ['IATA_CODE', 'AIRLINE']
            else: 
                required_columns = ['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']
            
            run_quality_gates_raw(df, base_name, required_columns)

            # Converte em parquet
            parquet_path = f"{stage_path}/{base_name}"
            df.write.mode("overwrite").option("compression", "snappy").parquet(parquet_path)
            
            log.info(f"[ConvertCSV] Arquivo convertido: {parquet_path}.")

        except Exception as e:
            log.error(f"[ConvertCSV][ERROR] Falha ao converter {csv}: {e}.")
            raise IOError(f"Erro ao processar {csv}: {e}.") from e


### Runner para o job `convert_csv_to_parquet`

In [ ]:
try:
    log.info("[ConvertCSV] Iniciando job de conversão de csv para parquet.")

    csv_files = check_files_in_folder(stage_path, "*.csv")
    target_files = [f for f in csv_files if "flights_part" not in f]

    if not target_files:
        raise FileNotFoundError(f"[ConvertCSV][Error] Nenhum arquivo csv com o padrão encontrado em {stage_path}.")

    convert_csv_to_parquet(spark, target_files, stage_path)

    log.info(f"[ConvertCSV] Conversão concluída. {len(target_files)} arquivo(s) processado(s).")

except Exception as e:
    log.exception(f"[ConvertCSV][Error] Falha durante execução: {e}.")
    raise

finally:
    log.info("[ConvertCSV] Job de conversão encerrado.")


In [ ]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[ConvertCSV] Sessão Spark finalizada.")


## Job 4: move_files_to_raw

Este job move os arquivos `parquet` da pasta **Stage** para a camada **Raw**, organizando-os por data de processamento.


In [ ]:
log = get_logger("move_to_raw")

spark = get_spark_session("MoveStageToRaw")
log.info("[MoveToRaw] SparkSession iniciada.")


### Runner para o job `move_files_to_raw`

In [ ]:
try:
    log.info("[MoveToRaw] Iniciando job de movimentação de arquivos.")

    parquet_files = check_files_in_folder(stage_path, "*.parquet")
    if not parquet_files:
        raise FileNotFoundError(f"[MoveToRaw][Error] Nenhum arquivo parquet encontrado em {stage_path}.")

    processing_date = datetime.now().strftime("%Y-%m-%d")

    move_files(
        spark=spark,
        source_files=parquet_files,
        base_dest_path=raw_path,
        processing_date=processing_date,
    )

    log.info(f"[MoveToRaw] {len(parquet_files)} arquivo(s) movido(s) para raw/{processing_date}.")

except Exception as e:
    log.exception(f"[MoveToRaw][Error] Falha durante execução: {e}.")
    raise

finally:
    log.info("[MoveToRaw] Job de movimentação encerrado.")


In [ ]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[MoveToRaw] Sessão Spark finalizada.")


## Job 5: cleanup_stage

Este job remove os arquivos `csv` e `parquet` da pasta **Stage** após a conclusão do carregamento na camada **Raw**.


In [ ]:
log = get_logger("cleanup_stage")

spark = get_spark_session("CleanupStage")
log.info("[CleanupStage] SparkSession iniciada.")


### Runner para o job `cleanup_stage`

In [ ]:
try:
    log.info("[CleanupStage] Iniciando job de limpeza da stage.")

    removable_files = check_files_in_folder(stage_path, "*.*")
    if not removable_files:
        log.warning("[CleanupStage] Nenhum arquivo encontrado para remoção.")
    else:
        delete_files(spark, removable_files)
        log.info(f"[CleanupStage] {len(removable_files)} arquivo(s) removido(s) da stage.")

except Exception as e:
    log.exception(f"[CleanupStage][Error] Falha durante execução: {e}.")
    raise

finally:
    log.info("[CleanupStage] Job de limpeza encerrado.")


In [ ]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[CleanupStage] Sessão Spark finalizada.")
